In [1]:
import os
import json
import pandas as pd

In [18]:
from datasets import load_dataset

data = load_dataset("Amod/mental_health_counseling_conversations")

In [19]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})


In [20]:
# preprocess the data for finetuning

import json

def preprocess_data(dataset):
    data = []
    for entry in dataset['train']:
        context = entry['Context']
        response = entry['Response']
        messages = [
            {"role": "user", "content": context},
            {"role": "assistant", "content": response}
        ]
        data.append({"messages": messages})
    return data

data_n = preprocess_data(data)

In [21]:
# save the preprocessed data to a JSON file

with open('fine_tune_data.jsonl','w') as f:
    for entry in data_n:
        f.write(json.dumps(entry) + '\n')

In [22]:
from collections import defaultdict

data_path = "fine_tune_data.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
  dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

Num examples: 3512
Found errors:
missing_content: 4


In [23]:
# Importing OpenAI 
import openai
from openai import OpenAI
import os

os.environ["OPENAI_API_KEY"] = "API_KEY"
openai.api_key=os.getenv('OPENAI_API_KEY')

client = OpenAI()

In [25]:
# fine tuning
# sending data to OpenAI

client.files.create(
    file = open('fine_tune_data.jsonl', 'rb'),
    purpose = 'fine-tune'
)

FileObject(id='file-BHKXHF3ZFBs9j3tNw3kQLqpF', bytes=4987663, created_at=1718070827, filename='fine_tune_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [26]:
client.fine_tuning.jobs.create(
    training_file='file-BHKXHF3ZFBs9j3tNw3kQLqpF',
    model='gpt-3.5-turbo'
)

FineTuningJob(id='ftjob-BIGkTrcQWT6Jt1loGndfi4AE', created_at=1718070877, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-WrpVCZI7H4QoDgRGsecsrDn2', result_files=[], seed=367529326, status='validating_files', trained_tokens=None, training_file='file-BHKXHF3ZFBs9j3tNw3kQLqpF', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [32]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-BIGkTrcQWT6Jt1loGndfi4AE', created_at=1718070877, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=7, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-WrpVCZI7H4QoDgRGsecsrDn2', result_files=[], seed=367529326, status='running', trained_tokens=None, training_file='file-BHKXHF3ZFBs9j3tNw3kQLqpF', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)], object='list', has_more=False)

In [29]:
system_prompt = "Jessy is a mental health councelling chatbot who likes to answer with bullets."
user_context = "I am a kidnapping survivor and have PTSD. I have an autoimmune disease which keeps on growing strong day by day."

In [31]:
# Check the status of the fine-tuning job
fine_tune_job_id = 'ftjob-BIGkTrcQWT6Jt1loGndfi4AE'  # Replace with your actual job ID
fine_tune_job = openai.FineTune.retrieve(id=fine_tune_job_id)

# Print the fine-tuning job details
print(fine_tune_job)

# Get the fine-tuned model ID
if fine_tune_job['status'] == 'succeeded':
    fine_tuned_model_id = fine_tune_job['fine_tuned_model']
    print(f"Fine-tuned model ID: {fine_tuned_model_id}")
else:
    print("Fine-tuning job is not yet completed or has failed.")

APIRemovedInV1: 

You tried to access openai.FineTune, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [30]:
response = client.chat.completions.create(
    model = "ft:gpt-3.5-turbo-0125",
    messages = [
        {'role':'system', "content": system_prompt},
        {'role':'user', "content": user_context}
    ]
)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `ft:gpt-3.5-turbo-0125` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}